# Домашнее задание №1.

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

Помимо изученных на данном вебинаре метрик можно отдельно выделить метрики качества ранжирования на основе ранговой корреляции, которая в отличие от обычной корреляции учитывает не сами значения, а лишь их ранг (порядок). Два наиболее распространенных ранговых коэффициентов корреляции: коэффициенты Спирмена и Кендэлла.   
Рассматривая модели поведения пользователей, можно выделить, так называемые, каскадные модели, согласно которым изучение предложенных пользователю элементов происходит последовательно и вероятность просмотра каждого последующего зависит от релевантности предыдущих. Здесь можно выделить две метрики: Expected reciprocal rank (ERR) и PFound.   
В случаях, когда информация о релевантности известна только для части элементов (документов, товаров), также может быть использована метрика Bpref.

**2) Доделать все функции, где стоит комментарий "сделать дома"**

In [1]:
import numpy as np

### Hit rate

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    return hit_rate(recommended_list[:k], bought_list)

In [3]:
recommended_list = [156, 1134, 27, 1543, 3345, 143, 533, 11, 43]
bought_list = [521, 32, 143, 991]

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
hit_rate_at_k(recommended_list, bought_list, k=5)

0

### Precision

In [6]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    
    return precision(recommended_list[:k], bought_list)


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()

In [7]:
recommended_list = [2, 3, 8, 6, 9, 4, 5, 10]
bought_list = [1, 5, 6]
prices = [23, 2233, 534, 56, 1]

In [8]:
precision(recommended_list, bought_list)

0.25

In [9]:
precision_at_k(recommended_list, bought_list, k=5)

0.2

In [10]:
money_precision_at_k(recommended_list, bought_list, prices, k=5)

0.01966982788900597

### Recall

In [11]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_bought, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(bought_list, recommended_list)
    
    return np.dot(flags, prices_bought).sum() / prices_bought.sum()

In [12]:
recommended_list = [2, 3, 8, 6, 9, 4, 5, 10]
bought_list = [1, 5, 6]
prices = [23, 2233, 534]

In [13]:
recall(recommended_list, bought_list)

0.6666666666666666

In [14]:
recall_at_k(recommended_list, bought_list, k=5)

0.3333333333333333

In [15]:
money_recall_at_k(recommended_list, bought_list, prices, k=5)

0.1913978494623656

In [16]:
money_recall_at_k(recommended_list, bought_list, prices, k=8)

0.9917562724014337

### AP@k
AP@k - average precision at k

In [17]:
recommended_list = [22, 21, 4, 1, 234, 232432, 234234, 666]
bought_list = [1, 221, 3, 4,]

In [18]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

In [19]:
ap_k(recommended_list, bought_list, k=5)

0.41666666666666663

### MAP@k

MAP@k (Mean Average Precision@k)

In [20]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]]

bought_list_3_users = [[521, 32, 143],
                       [143, 156, 991, 4, 1],
                       [27, 43]]

In [21]:
def map_k(recommended_list, bought_list, k=5):
    
    users_amount = len(bought_list)
    
    sum_ = sum([ap_k(recommended_list[i], bought_list[i], k) for i in range(users_amount)])
    
    return sum_ / users_amount

In [22]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.5388888888888889

### Normalized discounted cumulative gain ( NDCG@k)

In [23]:
def dcg_at_k(recommended_list, bought_list, k=5):
    
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    bought_facts = np.isin(recommended_list, bought_list)
    discounts = np.array([np.log2(i + 2) for i in range(k)])
    
    return (bought_facts / discounts).sum()


def ndcg_at_k(recommended_list, bought_list, k=5):
    
    return dcg_at_k(recommended_list, bought_list, k) / dcg_at_k(recommended_list, recommended_list[:k], k)

In [24]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.3156484524795145

### Mean Reciprocal Rank ( MRR@k )

In [25]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    return 0 if len(relevant_indexes) == 0 else (1 / (relevant_indexes[0] + 1))


def mean_reciprocal_rank(users_recommended_list, users_bought_list, k=5):
    
    size = len(users_bought_list)
    
    return np.array([reciprocal_rank(users_recommended_list[i], users_bought_list[i], k) for i in range(size)]).mean()

In [26]:
mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=3)

0.4444444444444444